In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
df_products_detailed = pd.read_csv("products_detailed.csv")
df_products_detailed.head()

,id,name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,is_natural_organic,is_natural_sephora,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_name,brand_id,brand_long
0,P448857,Apple Cider Vinegar Detangling Conditioner,"An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine.","What it is: An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine. Hair Type: Straight, Wavy, Curly, and Coily Hair Texture: Fine, Medium, and Thick Key Benefits: Detangles, Adds Shine, and Prevents Damage Formulation: Liquid Highlighted Ingredients: - Sunflower Seed Extract: Protects hair color. - Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Ingredient Callouts: Free of sulfates SLS and SLES, parabens, and phthalates. This product is also vegan, cruelty-free, and gluten-free, and comes in recyclable packaging. What Else You Need to Know: This conditioner is formulated to detangle hair and provide an extra conditioning boost after cleansing hair. It is infused with conditioning agents argan oil, aloe vera, and lavender extract to nurture hair and sunflower seed extract to protect hair color.",2276269,6.5 oz/ 192 mL,30.00,None,NaN,30.00,30.00,0,0,0,2.80,5,280,"{'cat60131': 'Conditioner', 'cat1230050': 'Shampoo & Conditioner', 'cat130038': 'Hair'}",https://www.sephora.com/product/apple-cider-vinegar-detangling-conditioner-P448857?skuId=2276269,https://www.sephora.com/productimages/sku/s2276269-main-zoom.jpg,"-Sunflower Seed Extract: Protects hair color. -Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Water/Aqua/Eau, Cetearyl Alcohol, Caprylic/Capric Triglyceride, Behentrimonium Chloride, Cetrimonium Chloride, Stearalkonium Chloride, Diheptyl Succinate, Capryloyl Glycerin/Sebacic Acid Copolymer, Polyquaternium-47, Hydrolyzed Vegetable Protein, Apple Cider Vinegar, Glycerin, Hydroxyethylcellulose, Argania Spinosa Kernel Oil, Macadamia Ternifolia Seed Oil, Spathodea Campanulata Flower Extract, Taraxacum Officinale (Dandelion) Leaf Extract, Lavandula Angustifolia (Lavender) Extract, Aloe Barbadensis Leaf Juice, Tocopheryl Acetate, Butylene Glycol, Helianthus Annuus (Sunflower) Seed Extract, Tetrasodium Glutamate Diacetate, Sodium Hydroxide, Phenoxyethanol, Ethylhexylglycerin, Fragrance (Parfum).","Suggested Usage: -Apply to clean, wet hair. -Massage through mid-lengths and ends of hair, then rinse thoroughly. -For best results, cleanse before with dpHUE Apple Cider Vinegar Hair Rinse.",dpHUE,6147,NaN
1,P425464,Surf Spray Mini,"A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look.","What it is: A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look. Hair Type: Straight and Wavy Hair Texture: Fine and Medium Hair Concerns: Hold and Volumizing Formulation: Spray Highlighted Ingredients: - Ascophyllum Nudosum (Seaweed) Extract: Extracted from naturally occurring seaweed; softens and smoothes skin and moisturizes hair, increasing its elasticity, luster, and softness; and helps decrease hair’s static charge, thus giving more body and less flyaways. - Macrocystis Pyrifera (Kelp) Extract: A natural seaweed extract with a high iodine and mineral content; contributes shine and manageability to hair and reduces the harshness of some chemicals; the nutritive value of this extract makes it ideal for skin preparations. - Magnesium Sulfate: Commonly known as Epsom salts; a natural mineral, which imparts astringent action. Ingredient Callouts: Free of parabens, formaldehydes, phthalates, and mineral oil. This product is also cruelty-free. What Else You Need to Know: This saltwater-based, st

In [3]:
df_products_detailed.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3538 entries, 0 to 3537
Data columns (total 25 columns):
id                    3538 non-null object
name                  3538 non-null object
short_desc            3538 non-null object
long_desc             3538 non-null object
item                  3538 non-null int64
item_name             3077 non-null object
list_price            3538 non-null float64
variation             3538 non-null object
variants              1584 non-null object
price_low             3538 non-null float64
price_high            3538 non-null float64
is_limited_edition    3538 non-null int64
is_natural_organic    3538 non-null int64
is_natural_sephora    3538 non-null int64
rating                3481 non-null float64
review_count          3538 non-null int64
user_favorites        3538 non-null int64
categories            3536 non-null object
url                   3538 non-null object
image_url             3538 non-null object
ingredients           3018 non-null

In [4]:
categories = []
for group in df_products_detailed[df_products_detailed.categories.isna()==False].categories.unique():
    group = json.loads(group.replace("\'", "\""))
    for key, val in group.items():
        if key:
            categories.append(val)
            
categories = list(set(categories))
len(categories), categories

(156,
 ['Face Brushes',
  'Other Needs',
  'Hair Tools',
  'Leave-In Conditioner',
  'Eyebrow',
  'Face Oils',
  'Hair Oil',
  'Value & Gift Sets',
  'Beauty Supplements',
  'Cleansers',
  'Anti-Aging',
  'Body Lotions & Body Oils',
  'Hair Styling Products',
  'Facial Rollers',
  'Eye Masks',
  'Face Wipes',
  'Eye',
  'Skincare',
  'Shampoo',
  'Mascara',
  'Tweezers & Eyebrow Tools',
  'Lip Balms & Treatments',
  'Eye Brushes',
  'Sheet Masks',
  'Body Sunscreen',
  'Bath Soaks & Bubble Bath',
  'Aftershave',
  'Fragrance',
  'Lip Stain',
  'For Face',
  'Lip Gloss',
  'Color Care',
  'Setting Spray & Powder',
  'Mists & Essences',
  'Makeup Palettes',
  'Cellulite & Stretch Marks',
  'Nail',
  'Diffusers',
  'False Eyelashes',
  'Makeup & Travel Cases',
  'Facial Peels',
  'Hair Removal',
  'Cleansing Brushes',
  'Lip',
  'Face Primer',
  'Bath & Body',
  'Scrub & Exfoliants',
  'Hair Masks',
  'Conditioner',
  'Makeup Removers',
  'Color Correct',
  'Hair Products',
  'Body Produc

In [5]:
df_category = df_products_detailed[["id", "categories"]].copy()

for i in range(len(df_category)):
    if df_category.categories.iloc[i] is np.nan:
        continue
    else:
        categories = json.loads(df_category.categories.iloc[i].replace("\'", "\""))
        for cat in categories.values():
            header=cat.lower().replace(" ", "_").replace("&", "and").replace("-", "_")
            df_category.at[i, header] = 1
        
display(list(df_category.columns))

['id',
 'categories',
 'conditioner',
 'shampoo_and_conditioner',
 'hair',
 'mini_size',
 'concealer',
 'face',
 'makeup',
 'value_and_gift_sets',
 'skincare',
 'moisturizers',
 'rollerballs_and_travel_size',
 'women',
 'fragrance',
 'blush',
 'cheek',
 'bath_and_shower',
 'men',
 'facial_cleansing_brushes',
 'high_tech_tools',
 'perfume',
 'cellulite_and_stretch_marks',
 'body_care',
 'bath_and_body',
 'shampoo',
 'eye_primer',
 'eye',
 'lipstick',
 'lip',
 'perfume_gift_sets',
 'face_primer',
 'false_eyelashes',
 'body_wash_and_shower_gel',
 'hair_styling_products',
 'hair_styling_and_treatments',
 'lip_gloss',
 'scrub_and_exfoliants',
 'face_masks',
 'masks',
 'eyeliner',
 'skincare_sets',
 'face_brushes',
 'brushes_and_applicators',
 'hair_straighteners_and_flat_irons',
 'hair_tools',
 'tools_and_brushes',
 'beauty_supplements',
 'wellness',
 'lip_sets',
 'face_wash_and_cleansers',
 'cleansers',
 'face_sets',
 'eye_creams_and_treatments',
 'eye_care',
 'highlighter',
 'lip_balm_and

In [6]:
df_category.drop(columns="categories", axis=1, inplace=True)
df_category.fillna(0, inplace=True)

In [7]:
for col in df_category.columns[1:]:
    df_category[col] = df_category[col].astype("int64")
df_category  

,id,conditioner,shampoo_and_conditioner,hair,mini_size,concealer,face,makeup,value_and_gift_sets,skincare,moisturizers,rollerballs_and_travel_size,women,fragrance,blush,cheek,bath_and_shower,men,facial_cleansing_brushes,high_tech_tools,perfume,cellulite_and_stretch_marks,body_care,bath_and_body,shampoo,eye_primer,eye,lipstick,lip,perfume_gift_sets,face_primer,false_eyelashes,body_wash_and_shower_gel,hair_styling_products,hair_styling_and_treatments,lip_gloss,scrub_and_exfoliants,face_masks,masks,eyeliner,skincare_sets,face_brushes,brushes_and_applicators,hair_straighteners_and_flat_irons,hair_tools,tools_and_brushes,beauty_supplements,wellness,lip_sets,face_wash_and_cleansers,cleansers,face_sets,eye_creams_and_treatments,eye_care,highlighter,lip_balm_and_treatment,for_body,self_tanners,leave_in_conditioner,mascara,foundation,nail,body_products,other_needs,shaving,bb_and_cc_cream,hair_oil,dry_shampoo,lotions_and_oils,bronzer,face_serums,treatments,hair_masks,face_sunscreen,sun_care,liquid_lipstick,hair_accessories,lip_liner,cologne,tinted_moisturizer,eye_palettes,deodorant_and_antiperspirant,hair_brushes_and_combs,anti_aging,hair_primers,facial_peels,body_lotions_and_body_oils,body_moisturizers,brush_cleaners,curling_irons,hand_cream_and_foot_cream,brush_sets,exfoliators,scalp_and_hair_treatments,holistic_wellness,blemish_and_acne_treatments,eye_cream,toners,sponges_and_applicators,setting_spray_and_powder,moisturizer_and_treatments,face_oils,facial_rollers,lip_balms_and_treatments,lip_treatments,sheet_masks,candles,candles_and_home_scents,value_sets,face_wash,eyebrow,tweezers_and_eyebrow_tools,accessories,eyeshadow,color_correct,aftershave,hair_spray,contour,eye_brushes,makeup_brushes_and_applicators,makeup_removers,night_creams,decollete_and_neck_creams,mists_and_essences,sunscreen,mirrors_and_sharpeners,eye_masks,body_mist_and_hair_mist,hair_thinning_and_hair_loss,cologne_gift_sets,hair_dryers,lip_brushes,teeth_whitening,bath_soaks_and_bubble_bath,makeup_bags_and_travel_cases,for_face,makeup_palettes,color_care,best_for,cheek_palettes,hair_products,body_sunscreen,lip_plumper,bb_and_cc_creams,body_sprays_and_deodorant,cleansing_brushes,eye_sets,deodorant_for_men,hand_wash_and_soap,lip_stain,hair_removal,diffusers,face_wipes,hair_removal_and_shaving,curls_and_coils,eyelash_curlers,makeup_and_travel_cases
0,P448857,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,P425464,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,P428606,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,P458504,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,P442109,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,

In [8]:
df_category.to_csv("category.csv", index=False)

In [9]:
urls = [
    "https://www.sephora.com/beauty/new-beauty-products?icid2=meganav_new_justarrived_viewall_link",
    "https://www.sephora.com/shop/gifts-for-her",
    "https://www.sephora.com/shop/gifts-for-men",
    "https://www.sephora.com/shop/gifts-for-them",
    "https://www.sephora.com/shop/gifts-for-teenage-girls",
    "https://www.sephora.com/shop/men",
    "https://www.sephora.com/shop/travel-size-toiletries",
    "https://www.sephora.com/shop/value-sets?icid2=meganav_gifts_valuesets_link",
    "https://www.sephora.com/shop/editors-picks-gifts?icid2=meganav_gifts_editorspicks_link",
    "https://www.sephora.com/shop/luxury-gifts?icid2=meganav_gifts_luxurygifts_link",
    "https://www.sephora.com/shop/exclusive-products?icid2=meganav_gifts_sephoraexclusives_us_link"
]

In [10]:
categories = [
    "just_arrived", "gifts_her", "gifts_men", "gifts_them", "gifts_teens",
    "for_men", "travel_size", "gift_sets", "editors_picks", "luxury_gifts",
    "sephora_exclusives"
]

In [11]:
for i in categories:
    if i in df_category.columns:
        print(i)

In [12]:
categories = dict(zip(categories, urls))
categories

{'just_arrived': 'https://www.sephora.com/beauty/new-beauty-products?icid2=meganav_new_justarrived_viewall_link',
 'gifts_her': 'https://www.sephora.com/shop/gifts-for-her',
 'gifts_men': 'https://www.sephora.com/shop/gifts-for-men',
 'gifts_them': 'https://www.sephora.com/shop/gifts-for-them',
 'gifts_teens': 'https://www.sephora.com/shop/gifts-for-teenage-girls',
 'for_men': 'https://www.sephora.com/shop/men',
 'travel_size': 'https://www.sephora.com/shop/travel-size-toiletries',
 'gift_sets': 'https://www.sephora.com/shop/value-sets?icid2=meganav_gifts_valuesets_link',
 'editors_picks': 'https://www.sephora.com/shop/editors-picks-gifts?icid2=meganav_gifts_editorspicks_link',
 'luxury_gifts': 'https://www.sephora.com/shop/luxury-gifts?icid2=meganav_gifts_luxurygifts_link',
 'sephora_exclusives': 'https://www.sephora.com/shop/exclusive-products?icid2=meganav_gifts_sephoraexclusives_us_link'}

In [44]:
def get_page(url):
    chrome_path = "/Users/valmadrid/Downloads/chromedriver"
    browser = webdriver.Chrome(executable_path=chrome_path)

    browser.get(url)

    sleep(randint(7, 10))

    try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
    except:
        pass

    sleep(randint(7, 10))

    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = 30
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(1)
        no_of_pagedowns -= 1

    sleep(5)

    soup = BeautifulSoup(browser.page_source)
    browser.quit()
    return soup


def get_products(soup):
    sephora_url = "https://www.sephora.com"
    product_list = []
    if len(soup.find_all("a", href=re.compile("/product/"))):
        for tag in soup.find_all("a", href=re.compile("/product/")):
            product_dict = {}
            product_dict["brand_name"] = tag.find(
                attrs={
                    "data-at": re.compile("sku_item_brand")
                }).get_text(strip=True)
            try:
                product_dict["product_ref"] = tag.attrs["data-uid"].split()[0]
            except:
                ref_pattern = re.compile(r"P[0-9]+")
                product_dict["product_ref"] = ref_pattern.findall(
                    tag.attrs["href"])[0]
            try:
                product_dict["product_item"] = tag.attrs["data-uid"].split(
                )[-1]
            except:
                product_dict["product_item"] = np.nan
            try:
                product_dict["product_name"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_name")
                    }).get_text(strip=True)
            except:
                product_dict["product_name"] = np.nan
            try:
                product_dict["product_price"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_price_list")
                    }).get_text(strip=True)
            except:
                product_dict["product_price"] = np.nan
            try:
                product_dict["product_image"] = sephora_url + tag.find(
                    attrs={
                        "src": re.compile("/productimages/")
                    }).get("src")
                if product_dict["product_item"] is np.nan:
                    item_pattern = re.compile("s[0-9]+")
                    product_dict["product_item"] = item_pattern.findall(
                        product_dict["product_image"])[0]
            except:
                product_dict["product_image"] = np.nan
            try:
                product_dict["product_link"] = sephora_url + tag.attrs["href"]
            except:
                product_dict["product_link"] = np.nan
            product_list.append(product_dict)

    return product_list

In [ ]:
categories_list = categories.copy()
for category_name, category_url in categories.items():
    product_list=[]
    for page in range(1,5):
        if page == 1:
            full_url = category_url+"?pageSize=300"
        else:
            full_url = category_url+"?pageSize=300&currentPage="+str(page)
        soup = get_page(full_url)
        products = get_products(soup)
        product_list.extend(products)
        if len(product_list)==0 or len(products)<300:
            break
        sleep(3)
    categories_list[category_name] = product_list
    sleep(randint(3,5))

In [ ]:
categories_list

In [ ]:
with open("categories_list.json", "w") as outfile:
    json.dump(categories_list, outfile, indent=2)

In [41]:
ref_pattern = re.compile(r"P[0-9]+")
item_pattern = re.compile(r"s[0-9]+")

# for cat, products in categories_list.items():
#     for i in products:
#         i["product_ref"] = ref_pattern.findall(i["product_link"])[0]
#         if i["product_image"] is not np.nan:
#             i["product_item"] = item_pattern.findall(i["product_image"])[0]